In [1]:
import torch

bool_tensor = torch.tensor([True, True, False])

# Option 1: Using torch.nonzero
int_indices = torch.nonzero(bool_tensor).squeeze()


print(int_indices)  # output: tensor([0, 1])


tensor([0, 1])


In [2]:
cd /data/wlsgur4011/part_assembly/src/pointnext

/data/wlsgur4011/part_assembly/src/pointnext


In [3]:
%matplotlib widget

In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"
from jhutil import show_point_clouds
import torch
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
import argparse
import yaml
import os
import sys
import logging
import wandb
from tqdm import tqdm
import torch
import torch.nn as nn
from torch import distributed as dist, multiprocessing as mp
from torch.utils.tensorboard import SummaryWriter
from torch_scatter import scatter
import torch.nn.functional as F
import warnings
import numpy as np
from sklearn.metrics import confusion_matrix
from collections import defaultdict, Counter

torch.backends.cudnn.benchmark = False
warnings.simplefilter(action='ignore', category=FutureWarning)
sys.path.insert(0, "/data/wlsgur4011/part_assembly/src/pointnext/examples")

from openpoints.models import build_model_from_cfg
from openpoints.models.layers import torch_grouping_operation, knn_point
from openpoints.loss import build_criterion_from_cfg
from openpoints.scheduler import build_scheduler_from_cfg
from openpoints.optim import build_optimizer_from_cfg
from openpoints.dataset import build_dataloader_from_cfg, get_class_weights, get_features_by_keys
from openpoints.transforms import build_transforms_from_cfg
from openpoints.utils import AverageMeter, ConfusionMatrix
from openpoints.utils import set_random_seed, save_checkpoint, load_checkpoint, resume_checkpoint, setup_logger_dist, \
    cal_model_parm_nums, Wandb, generate_exp_directory, resume_exp_directory, EasyConfig, dist_utils, find_free_port
from openpoints.models.layers import furthest_point_sample



[2023-07-14 15:56:16] [DEBU] Popen(['git', 'version'], cwd=/data/wlsgur4011/part_assembly/src/pointnext, universal_newlines=False, shell=None, istream=None)
[2023-07-14 15:56:16] [DEBU] Popen(['git', 'version'], cwd=/data/wlsgur4011/part_assembly/src/pointnext, universal_newlines=False, shell=None, istream=None)
[2023-07-14 15:56:16] [DEBU] Trying paths: ['/root/.docker/config.json', '/root/.dockercfg']
[2023-07-14 15:56:16] [DEBU] No config file found


ModuleNotFoundError: No module named 'shortuuid'

In [ ]:
import sys
sys.argv = ["examples/segmentation/main.py" ,"--cfg" ,"cfgs/part_assembly/pointnext-l.yaml"]

In [ ]:
def get_config():
    parser = argparse.ArgumentParser('Scene segmentation training/testing')
    parser.add_argument('--cfg', type=str, required=True, help='config file')
    parser.add_argument('--profile', action='store_true', default=False, help='set to True to profile speed')
    args, opts = parser.parse_known_args()
    cfg = EasyConfig()
    cfg.load(args.cfg, recursive=True)
    cfg.update(opts)  # overwrite the default arguments in yml
    
    if cfg.seed is None:
        cfg.seed = np.random.randint(1, 10000)
    
    # init distributed env first, since logger depends on the dist info.
    cfg.rank, cfg.world_size, cfg.distributed, cfg.mp = dist_utils.get_dist_info(cfg)
    cfg.sync_bn = cfg.world_size > 1
    
    # init log dir
    cfg.task_name = args.cfg.split('.')[-2].split('/')[-2]  # task/dataset name, \eg s3dis, modelnet40_cls
    cfg.cfg_basename = args.cfg.split('.')[-2].split('/')[-1]  # cfg_basename, \eg pointnext-xl
    tags = [
        cfg.task_name,  # task name (the folder of name under ./cfgs
        cfg.mode,
        cfg.cfg_basename,  # cfg file name
        f'ngpus{cfg.world_size}',
        f'seed{cfg.seed}',
    ]
    opt_list = []  # for checking experiment configs from logging file
    for i, opt in enumerate(opts):
        if 'rank' not in opt and 'dir' not in opt and 'root' not in opt and 'pretrain' not in opt and 'path' not in opt and 'wandb' not in opt and '/' not in opt:
            opt_list.append(opt)
    cfg.root_dir = os.path.join(cfg.root_dir, cfg.task_name)
    cfg.opts = '-'.join(opt_list)
    
    cfg.is_training = cfg.mode not in ['test', 'testing', 'val', 'eval', 'evaluation']
    if cfg.mode in ['resume', 'val', 'test']:
        resume_exp_directory(cfg, pretrained_path=cfg.pretrained_path)
        cfg.wandb.tags = [cfg.mode]
    else:
        generate_exp_directory(cfg, tags, additional_id=os.environ.get('MASTER_PORT', None))
        cfg.wandb.tags = tags
    os.environ["JOB_LOG_DIR"] = cfg.log_dir
    cfg_path = os.path.join(cfg.run_dir, "cfg.yaml")
    with open(cfg_path, 'w') as f:
        yaml.dump(cfg, f, indent=2)
        os.system('cp %s %s' % (args.cfg, cfg.run_dir))
    cfg.cfg_path = cfg_path
    
    # wandb config
    cfg.wandb.name = cfg.run_name
    
    if cfg.model.get('in_channels', None) is None:
        cfg.model.in_channels = cfg.model.encoder_args.in_channels
        
    return cfg

cfg = get_config()
set_random_seed(cfg.seed + cfg.rank, deterministic=cfg.deterministic)
torch.backends.cudnn.enabled = True

launch mp with 1 GPUs, current rank: 0


## model & data_loader

In [ ]:
val_loader = build_dataloader_from_cfg(cfg.get('val_batch_size', cfg.batch_size),
                                       cfg.dataset,
                                       cfg.dataloader,
                                       datatransforms_cfg=cfg.datatransforms,
                                       split='val',
                                       distributed=cfg.distributed
                                    )

[2023-06-01 15:48:36] [INFO] 
Totally 17686 samples in val set


In [ ]:
for data in val_loader:
    break

In [ ]:

import jhutil; jhutil.jhprint(1111, data)

1111  {
    "pos": "tensor[1, 29751, 3] n=89253 (0.3Mb) x∈[-1.912, 5.608] μ=0.942 σ=1.797",
    "x": "tensor[1, 29751, 3] n=89253 (0.3Mb) x∈[-8.267, 2.856] μ=-2.607 σ=3.111",
    "y": "tensor[1, 29751] i64 0.2Mb x∈[0, 1] μ=0.137 σ=0.344",
    "heights": "tensor[1, 29751, 1] 0.1Mb x∈[0., 5.608] μ=2.827 σ=1.612"
}


In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [ ]:
from jhutil import show_point_clouds

pos = data["pos"][0]
y = data["y"][0]

show_point_clouds([pos[y == 0], pos[y == 1]])